In [1]:
!pip install -q torch gtts moviepy mutagen opencv-python accelerate git+https://github.com/huggingface/diffusers
!pip -q install diffusers["torch"] transformers
!pip -q install datasets

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [1]:
import os
import torch
# from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM, #for automatically loading pre trained model for casual language modelling
    AutoTokenizer, # automatically selecting corresponding valid tokenizer
    # BitsAndBytesConfig, # for quantization configurations
    # HfArgumentParser, #simplifies handling cmd line args for hugging face
    # TrainingArguments, # managing hyperparameters like learning rate, epochs, etc.
    pipeline, # easy-to-use interface for performing various NLP tasks, such as text generation, text classification, question answering, etc.
    # logging, # setting & managing logs. Useful for debug and tracking training.
)
from gtts import gTTS
from moviepy.editor import VideoClip, ImageClip, AudioClip, vfx, concatenate_videoclips, concatenate_audioclips, CompositeAudioClip
import cv2
from mutagen.wave import WAVE

In [2]:
system_prompt = "You are a creative writer. Generate engaging short stories based on the provided prompts. Each story should have a clear beginning, middle, and end, and should be imaginative and well-written."
model_1 = "NousResearch/Llama-2-7b-chat-hf"
model_2 = "kulkarni-atharva/Llama-2-7b-storyteller-v1-finetune"
prompt = "A story about hare and the tortoise"

In [3]:
model = AutoModelForCausalLM.from_pretrained(model_2)
tokenizer = AutoTokenizer.from_pretrained(model_2, trust_remote_code=True)

pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=800)
result = pipe(f"<s> [INST] <<SYS>> \n {system_prompt} \n <</SYS>> \n \n {prompt} [/INST]")
generated_paragraph_1 = result[0]['generated_text']
print(generated_paragraph_1)

c:\Users\Atharva Kulkarni\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


model-00001-of-00002.safetensors:   3%|2         | 273M/9.98G [00:00<?, ?B/s]

KeyboardInterrupt: 

In [ ]:
generated_paragraph = generated_paragraph_1.replace("<newline>", "")

In [ ]:
generated_prompts = generated_paragraph.split('.')

# prompt = f"generate prompts for each line of the following paragraph so that it can be fed to image generation model : {generated_paragraph}"
# pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=512)
# result = pipe(f"<s>[INST] \n {prompt} [/INST]")
# generated_prompts = result[0]['generated_text']
# generated_prompts = generated_prompts.split(".")

In [ ]:
audio_clips = []
audio_lengths = []
language="en"
# j = 1
# generated_paragraph = generated_paragraph.split('.')
output_directory = "./audio_files/"

# Create the directory if it doesn't exist
if not os.path.exists(output_directory):
    os.makedirs(output_directory)

for i, sentence in enumerate(generated_prompts):
    speech = gTTS(text=sentence, lang=language, slow=False, tld="com.au")
    filename = os.path.join(output_directory, f"speech_{i}.mp3")
    speech.save(filename)
    audio_clips.append(filename)
    audio = WAVE(filename)
    audio_info = audio.info
    length = int(audio_info.length)
    audio_lengths.append(length)

In [ ]:
for i in range(1,len(generated_prompts)):
    generated_prompts[i] = generated_prompts[0] + generated_prompts [i]
generated_images = []
from diffusers import StableDiffusionPipeline
pipe = StableDiffusionPipeline.from_pretrained("redstonehero/cetusmix_v4", torch_dtype=torch.float16)
pipe = pipe.to("cuda")
pipe.safety_checker = None
h=512
w=512
steps=50
guidance=7.5
neg = "easynegative, lowres, bad anatomy, bad hands, text, error, missing fingers, extra digit, fewer digits, cropped, worstquality, low quality, normal quality, jpegartifacts, signature, watermark, username, blurry, bad feet, cropped, poorly drawn hands, poorly drawn face, mutation, deformed, worst quality, low quality, normal quality, jpeg artifacts, signature, watermark, extra fingers, fewer digits, extra limbs, extra arms,extra legs, malformed limbs, fused fingers, too many fingers, long neck, cross-eyed,mutated hands, polar lowres, bad body, bad proportions, gross proportions, text, error, missing fingers, missing arms, missing legs, extra digit, extra arms, extra leg, extra foot,"
output_directory = "./images/"
os.makedirs(output_directory)
for i, prompt in enumerate(generated_prompts):
  prompt += "8K, ultradetailed"
  image = pipe(prompt, height=h, width=w, num_inference_steps=steps, guidance_scale=guidance, negative_prompt=neg).images[0]
  filename = os.path.join(output_directory , f"image_{i}.jpg")
  image.save(filename)
  generated_images.append(filename)

In [ ]:


# for sentence in generated_paragraph:
#     speech = gTTS(text=sentence,lang=language,slow=False,tld="com.au")
#     audio_clips.append(speech)
    # audio = WAVE(speech)
    # contains all the metadata about the wavpack file
    # audio_info = audio.info
    # length = int(audio_info.length)
    # audio_lengths.append(length)
    # speech.save(f"audio{j}.mp3")
    # j+=1

In [ ]:
video_clips = []
audio_clips_r = []
for i, image in enumerate(generated_images):
    clip = ImageClip(image).set_duration(audio_lengths[i])
    video_clips.append(clip)
for path in audio_clips:
  clip = AudioClip(path)
  audio_clips_r.append(clip)
final_video = concatenate_videoclips(video_clips)
# final_audio = concatenate_audioclips(audio_clips)
final_video.audio = CompositeAudioClip(audio_clips)
os.makedirs("./video/")
final_video.write_videofile("./video/final_video.mp4")